In [1]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import load
from sklearn.metrics import classification_report

<Figure size 1000x600 with 0 Axes>

In [2]:
wesad = load.WESAD()
df = wesad.separate_and_feature_extract(sample_n=420000) #700 * 60 * 10 (10mins)

Using ready-made DataFrame


In [3]:
df.shape

(30, 59, 20)

In [4]:
y = df[:, 0, 0]  # 取得 'label'
X = df[:, :, 2:]  # 移除 'label','subject'

In [5]:
y=y-1 #1,2->0,1

In [6]:
X = np.array(X, dtype=np.float32)

y = np.array(y, dtype=np.int32)

print("X dtype:", X.dtype)  # 應該是 float32
print("y dtype:", y.dtype)  # 應該是 int32

X dtype: float32
y dtype: int32


In [7]:
# 分割訓練測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [58]:
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

In [59]:
def build_cnn_lstm_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.LSTM(32, return_sequences=False))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

In [60]:
input_shape = (X_train.shape[1], X_train.shape[2]) #(59,18)
num_classes = len(np.unique(y_train))

print(input_shape)
print(num_classes)

(59, 18)
2


In [61]:
# 初始化模型
model = build_cnn_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 30
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/30


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5714 - loss: 1.1197 - val_accuracy: 0.6667 - val_loss: 1.5047
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.4286 - loss: 2.6382 - val_accuracy: 0.6667 - val_loss: 0.9551
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4286 - loss: 1.6227 - val_accuracy: 0.3333 - val_loss: 0.7466
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5714 - loss: 0.6668 - val_accuracy: 0.3333 - val_loss: 1.7310
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5714 - loss: 1.1174 - val_accuracy: 0.3333 - val_loss: 1.7824
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5714 - loss: 1.1555 - val_accuracy: 0.3333 - val_loss: 1.1981
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5714 - loss: 0.8362 - val_accuracy: 0.4444 - val_loss: 0.6816
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6667 - loss: 0.6388 - val_accuracy: 0.6667 - val_loss: 0.6432
Epoch 9/30
1/

In [62]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
              precision    recall  f1-score   support

           1       0.25      0.33      0.29         3
           2       0.60      0.50      0.55         6

    accuracy                           0.44         9
   macro avg       0.42      0.42      0.42         9
weighted avg       0.48      0.44      0.46         9



In [63]:
# 初始化模型
model = build_cnn_lstm_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 30
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/30


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5714 - loss: 0.6933 - val_accuracy: 0.3333 - val_loss: 0.7473
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5714 - loss: 0.6821 - val_accuracy: 0.3333 - val_loss: 0.7283
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5714 - loss: 0.6774 - val_accuracy: 0.3333 - val_loss: 0.7426
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5714 - loss: 0.6716 - val_accuracy: 0.3333 - val_loss: 0.7342
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5714 - loss: 0.6613 - val_accuracy: 0.4444 - val_loss: 0.7258
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5714 - loss: 0.6479 - val_accuracy: 0.4444 - val_loss: 0.7372
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6190 - loss: 0.6300 - val_accuracy: 0.5556 - val_loss: 0.6648
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7143 - loss: 0.6319 - val_accuracy: 0.4444 - val_loss: 0.8007
Epoch 9/30
1/

In [64]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
              precision    recall  f1-score   support

           1       0.33      0.33      0.33         3
           2       0.67      0.67      0.67         6

    accuracy                           0.56         9
   macro avg       0.50      0.50      0.50         9
weighted avg       0.56      0.56      0.56         9



In [65]:
# 選擇 LSTM 是因為 生理訊號屬於時間序列數據，LSTM 能夠記住過去的資訊，並利用這些資訊影響未來的預測。
# 此外，LSTM 透過 記憶單元與門機制，有效解決了傳統 RNN 中的 梯度消失問題，使模型能夠學習並捕捉長時間的依賴關係，從而提升壓力偵測的準確性。